In [1]:
from sklearn.linear_model import Ridge
import joblib
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../datasets/train.csv')
data.head()

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [3]:
X_train_roberta = joblib.load('roberta_mlm_cv_train/roberta_large_tr_p.pkl')
X_train_alberta = joblib.load('albert_mlm_cv_train/albert_xxlarge_v2_tr_p.pkl')
X_train_stacking = np.concatenate((X_train_roberta, X_train_alberta), axis=1)

In [4]:
ridge_reg = Ridge(alpha=1.0, max_iter=1000)  # 设置alpha=1.0(Regularization strength)
ridge_reg.fit(X_train_stacking, data['target'].values)

Ridge(max_iter=1000)

In [5]:
X_test_roberta = joblib.load('roberta_mlm_cv_train/roberta_large_te_pr.pkl').reshape(7, -1).mean(axis=1)
X_test_alberta = joblib.load('albert_mlm_cv_train/albert_xxlarge_v2_te_pr.pkl').reshape(7, -1).mean(axis=1)
X_train_stacking = np.stack((X_test_roberta, X_test_alberta), axis=1)

In [6]:
ridge_reg.predict(X_train_stacking)  # stacking模型融合

array([-0.94934314, -0.9695848 , -0.9621951 , -0.9472326 , -0.9673928 ,
       -0.95723575, -0.941799  ], dtype=float32)

In [7]:
X_test_roberta * 0.5 + X_test_alberta * 0.5  # 线性加权融合(可通过optuna包搜索各模型权重(超参数))

array([-1.2240238 , -0.501619  , -1.1376828 , -1.3129988 , -0.64898777,
       -0.8965702 , -1.1893468 ], dtype=float32)